<h1>langchain tests

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "381001efce9a4f468afacd97aba385c4"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["AZURE_OPENAI_ENDPOINT"]="https://lucasaiservices.openai.azure.com"

In [3]:
from langchain_openai import AzureOpenAI
#from openai import AzureOpenAI

# Create an instance of Azure OpenAI
# Replace the deployment name with your own
llm = AzureOpenAI(deployment_name="davinci-002")

ModuleNotFoundError: No module named 'langchain_openai'

In [ ]:
llm.invoke("how can langsmith help with testing?")

' Can you trust your test tool vendor?\n\nThe answer is yes, you can trust Langsmith. And the reason is the team behind Langsmith.\n\nLangsmith’s founders and developers are experienced testing experts who are very familiar with the issues that a test tool faces when being applied to a real-life testing project. They know that testing is not an isolated activity that can be done in a vacuum. Testing is an integral part of the project, and it is directly linked to the customer’s needs and the project’s goals.\n\nLangsmith’s developers have used the software they create and they are still using it today. They are actively involved in the testing community, sharing their experience and knowledge with the community. The Langsmith team has also developed a proprietary testing framework that is based on their real-life experience and expertise.\n\nThere are many test tools on the market today, but there is no other test tool that is better than Langsmith. The software is easy to use and the 

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4", openai_api_key="381001efce9a4f468afacd97aba385c4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

APIConnectionError: Connection error.

<h1>result object check

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

def initialize_kernel(model="gpt-35-turbo"):
    kernel = sk.Kernel()
    
    #deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    if model == "gpt-4":
        deployment = "gpt-4"
        endpoint = "https://lucasaiservices.openai.azure.com/openai/deployments/gpt-4/chat/completions?api-version=2023-03-15-preview"
    elif model == "gpt-4-2":
        deployment = "gpt-4-2"
        endpoint = "https://lucasaiservices.openai.azure.com/openai/deployments/gpt-4-2/chat/completions?api-version=2023-03-15-preview"
    else:
        deployment = "gpt-35-turbo"
        endpoint = "https://lucasaiservices.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2024-02-15-preview"
    
    api_key = "381001efce9a4f468afacd97aba385c4"
    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(service_id=service_id,
                            deployment_name=deployment,
                            endpoint=endpoint,
                            api_key=api_key)
    )
    return kernel


def initialize_functions(kernel):
    function_dict = {}
    
    # load in all functions
    plugins_directory = "./plugins"

    #functies voor interface
    function_list = kernel.import_plugin_from_prompt_directory(plugins_directory, "OutlinePlugin")
    function_dict["brainstorm"] = function_list["BrainstormOutline"]
    function_dict["check"] = function_list["BrainstormCheck"]
    function_dict["create_title"] = function_list["CreateTitle"]
    function_dict["outline"] = function_list["Outline"]
    function_dict["outline_split"] = function_list["OutlineSplit"]
    function_dict["json_create"] = function_list["OutlineJson"]
    
    
    #functies voor document generation
    function_list = kernel.import_plugin_from_prompt_directory(plugins_directory, "WebSearchPlugin")
    function_dict["query"] = function_list["CreateQuery"]
    function_dict["search_check"] = function_list["SearchCheck"]
    function_dict["public_search"] = kernel.import_native_plugin_from_directory(plugins_directory+"/WebSearchPlugin", "PublicSearch")["WebSearch"]
    function_dict["write_section"] = kernel.import_plugin_from_prompt_directory(plugins_directory, "WriterPlugin")["WriteSection"]
    
    #plugins_directory = "./plugins/ResearchPlugin"
    #function_list = kernel.import_native_plugin_from_directory(plugins_directory, "Websearch")
    #f = function_list["QueryToResults"]
    
    return function_dict

kernel = initialize_kernel()
function_dict = initialize_functions(kernel)

In [ ]:
result = await kernel.invoke(function_dict["brainstorm"], history="lol")

C:\Users\lcornette001\AppData\Local\Temp\ipykernel_11956\2226693672.py:1: RuntimeWarning: coroutine 'generate_document' was never awaited
  result = await kernel.invoke(function_dict["brainstorm"], history="lol")


In [ ]:
result.function.value

AttributeError: 'KernelFunctionMetadata' object has no attribute 'value'

<h1>VPN error check en rate limit error check

In [ ]:
e = """Error occurred while invoking function WriteSection: Error occurred while invoking function WriteSection: ("<class 'semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion.AzureChatCompletion'> service failed to complete the prompt", RateLimitError("Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded token rate limit of your current AIServices S0 pricing tier. Please retry after 3 seconds. Please contact Azure support service if you would like to further increase the default rate limit.'}}"))
Error occurred while invoking function: 'WriterPlugin.WriteSection'"""

In [ ]:
import re
match = re.search(r'Please retry after (\d+) seconds', e)

In [ ]:
int(match.group(1))

3

In [ ]:
import asyncio

In [ ]:
async def _call_api_with_rate_limit_handling(kernel, function, arguments):
    def _extract_retry_seconds(error_message):
        # Use regex to find the number of seconds to wait
        print(re.search(r'Please retry after (\d+) seconds', error_message))
        match = re.search(r'Please retry after (\d+) seconds', error_message)
        if match:
            return int(match.group(1))  # Return the number of seconds as an integer
        return None  # Return None if no match is found
    
    try:
        # Attempt to call the API
        response = await kernel.invoke(function, arguments)
        return response
    
    except Exception as e:
        wait_time = _extract_retry_seconds(e)
        if wait_time is not None:
            print(f"Rate limit exceeded, waiting for {wait_time} seconds...")
            await asyncio.sleep(wait_time)
            return await _call_api_with_rate_limit_handling(kernel, function, arguments)
        else:
            raise e

In [ ]:
import io
import re
import json
import asyncio
from ast import literal_eval
import semantic_kernel as sk
from dotenv import load_dotenv


from utils.document_handling import get_input_file, fill_in_front_page, write_document


async def _call_api_with_rate_limit_handling(kernel, function, arguments):
    def _extract_retry_seconds(error_message):
        # Use regex to find the number of seconds to wait
        match = re.search(r'Please retry after (\d+) seconds', error_message)
        if match:
            return int(match.group(1))  # Return the number of seconds as an integer
        return None  # Return None if no match is found
    
    try:
        # Attempt to call the API
        response = await kernel.invoke(function, arguments)
        return response
    
    except Exception as e:
        # Check if the exception has specific attributes you expect from the SDK documentation
        error_message = ""
        if hasattr(e, 'message'):
            print("eerste if")
            error_message = e.message
            print(error_message)
        elif hasattr(e, 'args') and len(e.args) > 0:
            print("tweede if")
            print(len(e.args))
            # Sometimes the message might be deeply nested in args
            print("arg 0: ", e.args[0])
        else:
            print("else")
            error_message = str(e)
            print(error_message)

        wait_time = _extract_retry_seconds(error_message)
        if wait_time is not None:
            print(f"Rate limit exceeded, waiting for {wait_time} seconds...")
            await asyncio.sleep(wait_time)
            return await _call_api_with_rate_limit_handling(kernel, function, arguments)
        else:
            raise e


async def _get_text(kernel, function_dict, prompt, outline):
    #search_check = str(await kernel.invoke(function_dict["search_check"], sk.KernelArguments(input=prompt)))
    #print(search_check)
    search_check = "False"
    
    if search_check == "True":
        query = str(await kernel.invoke(function_dict["query"], sk.KernelArguments(input=prompt, outline=outline)))
        search_results = str(await kernel.invoke(function_dict["public_search"], sk.KernelArguments(input=query)))
    else:
        search_results = ""
    
    if len(str(search_results)) > 16000:
        search_results = str(search_results[:16000])
    
    text = str(await _call_api_with_rate_limit_handling(kernel, function_dict["write_section"], sk.KernelArguments(input=prompt, searchResults=search_results)))
    
    return text


### Process each chapter
async def _fill_in_outline(kernel, function_dict, section, outline): # recursieve functie die doorheen prompts gaat en telkens de prompts naar tekst omzet
    title = section["title"]
    
    content=[]
    for prompt in section["prompts"]:
        content.append(await _get_text(kernel, function_dict, prompt, outline))
    
    sections=[]
    for subsection in section["sections"]:
        sections.append(await _fill_in_outline(kernel, function_dict, subsection, outline))
    
    return {"title":title, "content":content, "sections":sections}


def _doc_to_bytes(doc):
    doc_bytes = io.BytesIO()  
    doc.save(doc_bytes)  
    doc_bytes.seek(0)
    
    return doc_bytes


async def _generate_document_bytes(kernel, function_dict, title, outline_json: dict):
    load_dotenv()
    # initiatie template document bovenhalen
    doc = get_input_file("./data/input_file.docx")

    # prompts naar gpt gegenereerde tekst omzetten
    content_json = {"sections":[]}
    # Create tasks for filling in each section outline
    fill_section_tasks = [asyncio.create_task(_fill_in_outline(kernel, function_dict, section, outline_json))
                          for section in outline_json["sections"]]
    
    # Await all tasks to complete and maintain order of sections
    filled_sections = await asyncio.gather(*fill_section_tasks)
    content_json["sections"].extend(filled_sections)

    # maak voorpagina voor document
    doc = fill_in_front_page(doc, title)
    # tekst in het document steken
    for section in content_json["sections"]:
        doc = write_document(doc, section)

    # bytes object voor download
    doc_bytes = _doc_to_bytes(doc)
    
    return doc_bytes


async def _create_section_json(kernel, function_dict, outline_section_string):
    # Create tasks for asynchronously processing each outline section to create JSON
    outline_section_json_string = str(await _call_api_with_rate_limit_handling(kernel, function_dict["json_create"], sk.KernelArguments(outline=outline_section_string)))
    
    try:
        # Attempt to load the JSON structure from the response
        outline_section_json = json.loads(outline_section_json_string)
    except json.JSONDecodeError:
        # Log any JSON decode errors
        print("Error decoding JSON:")
        print(outline_section_json_string)
        outline_section_json = json.loads(outline_section_json_string[7:-3])
    
    return outline_section_json


async def _create_outline_json(kernel, function_dict, history):
    # Retrieve the definitive outline string
    outline_string = str(await kernel.invoke(function_dict["outline"], history=history))
    
    # Convert outline string to a list of outline strings
    outline_list_string = str(await kernel.invoke(function_dict["outline_split"], outline=outline_string))
    
    try:
        # Try to evaluate the outline list to a Python list object
        outline_list = literal_eval(outline_list_string)
    except SyntaxError:
        # Handle potential issues with string literals
        outline_list = outline_list_string.replace("\"", "\"\"\"")
        outline_list = literal_eval(outline_list_string)
    
    # Prepare a list of tasks
    tasks = [_create_section_json(kernel, function_dict, outline_section_string) for outline_section_string in outline_list]
    
    # Run tasks concurrently and wait for all to complete
    sections = await asyncio.gather(*tasks)
    
    # Collect results into outline_json
    outline_json = {"sections": sections}
    
    # maak titel
    title = str(await kernel.invoke(function_dict["create_title"], outline=outline_string))
    
    return title, outline_json


async def generate_document(kernel, function_dict, history):
    title, outline_json = await _create_outline_json(kernel, function_dict, history)
    doc_bytes = await _generate_document_bytes(kernel, function_dict, title, outline_json)
    return title, doc_bytes



In [ ]:
result = await generate_document(kernel, function_dict, history="""Introduction

Brief overview of the pharmaceutical industry
Importance and impact of the industry on global health and economy
Trends in the Pharma Industry

Shift towards personalized medicine
Increasing focus on biotechnology and biopharmaceuticals
Embracing digital health technologies
Regulatory changes and their impact on the industry
The Position of China and India in Pharma

Overview of the pharmaceutical markets in China and India
Growth and development of the industry in both countries
Challenges and opportunities for pharmaceutical companies operating in China and India
Overview of M&A Activity in Pharma

Recent mergers and acquisitions in the pharmaceutical industry
Impact of M&A on the competitive landscape
Regulatory considerations and challenges in M&A activity
Status of Biopharma

Definition and significance of biopharmaceuticals
Growth and market trends in the biopharma sector
Regulatory and manufacturing challenges in biopharmaceutical development
Biggest Biopharma Players

Profiles of leading biopharmaceutical companies
Their contributions to the industry and global healthcare
Strategies for success and future outlook
Biopharma Clusters in Europe and Globally

Overview of major biopharma clusters in Europe
Comparison with biopharma clusters in other regions
Factors contributing to the growth of biopharma clusters
Trends in Clinical Trials

Evolution of clinical trial methodologies
Adoption of virtual and decentralized clinical trials
Impact of real-world evidence on clinical research
The Impact of AI on Pharma

Applications of artificial intelligence in drug discovery and development
AI-driven approaches to personalized medicine
Regulatory and ethical considerations in AI adoption in pharma
This outline covers the main chapters and subtopics you requested. Let me know if you'd like to add more details or if there's anything else you'd like to include!""")

C:\Users\lcornette001\AppData\Local\Temp\ipykernel_11956\1953984981.py:1: RuntimeWarning: coroutine 'generate_document' was never awaited
  result = await generate_document(kernel, function_dict, history="""Introduction


In [ ]:
str(e)

'Error occurred while invoking function WriteSection: Error occurred while invoking function WriteSection: ("<class \'semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion.AzureChatCompletion\'> service failed to complete the prompt", RateLimitError("Error code: 429 - {\'error\': {\'code\': \'429\', \'message\': \'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded token rate limit of your current AIServices S0 pricing tier. Please retry after 3 seconds. Please contact Azure support service if you would like to further increase the default rate limit.\'}}"))\nError occurred while invoking function: \'WriterPlugin.WriteSection\''

<h1>autogen

In [12]:
import os
from autogen import AssistantAgent, UserProxyAgent

config_list = {
"model": "gpt-35-turbo",
"azure_endpoint": "https://lucasaiservices.openai.azure.com/",
"api_type": "azure",
"api_version": "2024-02-15-preview",
}

llm_config = config_list
assistant = AssistantAgent("assistant", llm_config=llm_config)
user_proxy = UserProxyAgent("user_proxy", code_execution_config=False)

# Start the chat
user_proxy.initiate_chat(
    assistant,
    message="Tell me a joke about NVDA and TESLA stock prices.",
)

user_proxy (to assistant):

Tell me a joke about NVDA and TESLA stock prices.

--------------------------------------------------------------------------------
assistant (to user_proxy):

I'm not aware of any specific jokes about NVDA and TESLA stock prices. However, I can create a simple financial joke for you. Here it is:

Why don't stock prices ever go to the doctor?
Because they prefer to avoid the "dow"!

If you'd like a specific joke related to NVDA and TESLA stock prices, I can help you create one based on their performance or any other specific details you have in mind. Just let me know!

--------------------------------------------------------------------------------


KeyboardInterrupt: Interrupted by user

In [14]:
import os
from autogen import AssistantAgent, UserProxyAgent
from autogen.coding import DockerCommandLineCodeExecutor

config_list = [
    {
"model": "gpt-35-turbo",
"azure_endpoint": "https://lucasaiservices.openai.azure.com/",
"api_type": "azure",
"api_version": "2024-02-15-preview",
}]

# create an AssistantAgent instance named "assistant" with the LLM configuration.
assistant = AssistantAgent(name="assistant", llm_config={"config_list": config_list})

# create a UserProxyAgent instance named "user_proxy" with code execution on docker.
code_executor = DockerCommandLineCodeExecutor()
user_proxy = UserProxyAgent(name="user_proxy", code_execution_config={"executor": code_executor})

DockerException: Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')

In [ ]:
from pypdf import PdfReader
from pprint import pprint

def extract_text_from_pdf(pdf_file):
    reader = PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    
    return text


In [4]:
text = extract_text_from_pdf("test.pdf")

In [5]:
pprint(text)

('   \n'
 ' \n'
 '  \n'
 ' \n'
 '1  \n'
 ' \n'
 ' \n'
 ' \n'
 'Project Title  Towards an Interoperable MARinE Knowledge GRAPH  \n'
 'Project Acronym  MareGraph  \n'
 'Grant Agreement No.  101100771   \n'
 'Start Date of Project  23/01/2022  \n'
 'Duration of Project  36 Months  \n'
 ' \n'
 'D2.2 Guidelines on process and methodology for \n'
 'organisational interoperability (Version 1)  \n'
 ' \n'
 'Work Package  WP 2, Process tuning, community building and dissemination  \n'
 'Lead Author (Org)  Lorenzo Vylders  (Digitaal Vlaanderen ) \n'
 'Contributing Author(s) \n'
 '(Org)  Lucas Cornette  (Digitaal Vlaanderen ), Liesbeth V an der H aegen '
 '(Digitaal \n'
 'Vlaanderen), Laurens Vercauteren (Digitaal Vlaanderen)  \n'
 'Due Date  30.06.2024 \n'
 'Date  29.01.2024 \n'
 'Version  V1.0  \n'
 ' \n'
 ' \n'
 'Dissemination Level  \n'
 'X PU: Public  \n'
 ' PP: Restricted to other programme participants (including the '
 'Commission)  \n'
 ' RE: Restricted to a group specified by the consor

In [1]:
from openai import AzureOpenAI

In [6]:
KEY = "07d376b4bbce4baf8a746f8be5437d0d"
ENDPOINT = "https://lucasaieastus2servicesopen.cognitiveservices.azure.com/"

client = AzureOpenAI(
        api_key=KEY,
        api_version="2023-12-01-preview",
        azure_endpoint=ENDPOINT,
    )

In [19]:
transcript = """0:00
When Russia launched its February 2022 invasion, one of its stated goals was the demilitarisation of Ukraine.
0:05
And indeed by June 2023, Russian state media ran a headline suggesting that goal had mostly been accomplished.
0:12
Shockingly enough however, more than a year after that headline the Russian military appears to still be locked in a grinding war
0:18
of attrition against the Ukrainian armed forces. Continuously applying firepower, manpower and materiel against Ukrainian defences
0:25
seemingly in order to slowly stretch, corrode and grind forward. It is an approach that has almost certainly destroyed far more Russian than Ukrainian equipment.
0:33
But despite the disparity, Ukrainian equipment losses have still been immense. And Russia certainly began the invasion with more equipment in hand.
0:40
It's a situation that raises a number of questions that I want to start looking at today. What have Ukrainian losses looked like? What sort of resupply have they received?
0:48
How is the force changing and is the equipment picture improving or degrading over time? I'll start by looking at the potential impact of equipment quality,
What Am I Talking About?
0:55
especially as Ukraine goes over to using more and more NATO equipment. And then start diving straight into questions of quantity.
1:02
That'll mean looking at both the big picture of how Ukrainian losses have generally compared to Russian ones so far, and what the wider resupply picture looks like.
1:09
But also some deep dives into specific Ukrainian equipment categories: tanks, infantry fighting vehicles, APCs and aircraft.
1:16
Note that in this episode I won't be focusing either on air defence or artillery systems because they are likely to get their own coverage in the future.
1:22
But I will close out with some broader observations about the way the Ukrainian army's equipment might be changing over time.
1:28
And perhaps more importantly, whether it has enough of it. But before I start hitting you with an array of graphs and stats a word from a sponsor.
1:36
And today I get to welcome back long-term channel sponsor and my VPN of choice, Private Internet Access.
1:41
Whether you're absorbing YouTube content or trawling through satellite imagery, our modern online environment certainly has a lot to offer.
1:48
But it can also feel like an environment that is steadily becoming more fragmented, and one with some potentially very rough edges
1:53
when it comes to things like personal privacy and data breaches. When trying to look out for your privacy in that sort of environment
1:59
you might want to make sure you have the right combination of tools and practices. Much like the seat belts and safety systems on an airplane,
2:05
much of the time you might be fine without them. But if something strange does happen you are probably going to be happy they are there.
2:11
In an online environment that might mean things like watching where you browse, keeping strong passwords and maintaining a healthy distrust of things like links and attachments.
2:19
And then if you're like me, you'll also want a VPN. With a click what a VPN allows you to do is reroute your internet traffic,
2:25
change your publicly perceived IP address, while also potentially changing how your traffic is logged. Noting that Private Internet Access has a no logs policy
2:33
that's reportedly been both audited and tested in court. They offer servers in 91 different countries,
2:39
plus if you want a little extra precision there are also servers in 50 US states. All in the context of software which is advertised as being open source,
2:47
available cross platform and compatible with a range of streaming services. Plus you probably won't have to worry about juggling multiple accounts,
2:53
because a single subscription will cover an unlimited number of devices. So if you are interested, there'll be a link in the description that will give you access
3:00
to an 83% discount on a 2 year subscription, plus 4 bonus months, covering an unlimited number of devices and with a 30 day money-back guarantee.
3:08
So with my genuine thanks to a long running sponsor, let's get back to it. OK, so a couple of weeks ago (at time of recording) we looked at some of the
Assessing Ukraine & Russia
3:15
available evidence for Russian heavy equipment losses in Ukraine, and tried to get an understanding of how deep their remaining reserves might go.
3:21
But in order to better understand where the balance of this war of attrition might drift long term, we probably have to repeat that exercise for the Ukrainians.
3:29
To a great extent, a lot of the harsh arithmetic of war here is going to be very similar. How much did the Ukrainians start with, how quickly
3:35
are they losing it compared to the invading Russians? And what does the pipeline look like in terms of both the quality and quantity of replacements?
3:42
But compared to that work on Russia, I think there's a couple of differences in Ukraine's situation that we have to try and reflect.
3:47
For example, comparatively speaking stored equipment is less of a concern for the Ukrainians than the Russians.
3:52
Compared to just about any other European state, Ukraine did begin the war with an immense amount of equipment in storage.
3:58
We are talking hundreds of tanks, airframes that could be reactivated, retired ammunition, etc.
4:03
And as you can see on the right there from that image (from I believe 2018) Ukrainian storage standards for some of that reserved equipment were almost Russian.
4:12
We've seen plenty of evidence that some of this equipment is being reactivated, both in terms of what we're seeing appear on the battlefields,
4:18
the announcements of the Ukrainian government, and also programs like the one you see on the right there. Where the Czech government announced that their industry would be assisting
4:25
the Ukrainians with reactivating old T-64s. If you want a very rough guesstimate for how many reserved main battle tanks
4:32
the Ukrainians have repaired or reactivated since February 2022, I'd put it in the low triple digits.
4:37
But compared to the Russian Federation, Ukraine has nowhere near the same breadth of stored equipment, nor the same reliance on it.
4:43
Ukraine doesn't have anything like Russia's stockpiles of old Soviet artillery systems for example.
4:48
Which means when we are modelling force endurance another factor has to come into play. Instead in Ukraine's case we have to put a much greater weighting on the importance of allied resupply.
4:58
This is obviously a factor for Russia too, especially when you are talking about things like drones from Iran or ammunition from North Korea.
5:04
But in Ukraine's case, what equipment is being donated and what equipment we expect them to receive in the future is a much more important part of the analysis.
5:11
For the purpose of today's episode, I am generally going to be counting any heavy equipment that has either been delivered or explicitly pledged as equipment that is supplied.
5:20
We are generally going to proceed on the assumption that Ukraine is going to receive everything that's been promised and nothing more.
5:25
I also won't be analysing NATO's equipment levels or storages this episode. Because frankly, with a couple of niche exceptions, the limitation on resupply to Ukraine
5:34
is almost always going to be a question of politics and priority more than raw availability. Across basically every equipment category, Ukraine's allies have access
5:42
to far, far more heavy equipment than Russia does. Colloquially we might refer to Russia and Ukraine as being low on equipment
5:48
once they are a couple of steps away from shovels and Molotov cocktails. Elsewhere, being "short" might mean not having a spare army or two's
5:54
worth of armoured vehicles parked in the desert like the United States. Or having some of the largest armoured forces in Europe
6:00
pointed at each other, as in the case of Greece and Türkiye. For Ukraine, often the question is not is there equipment out there,
6:06
it's whether they'll be given access to it. The other thing that sets Ukraine's equipment position apart is the unique role
6:13
played by arguably its greatest equipment donor, the Russian Federation. Captured equipment plays a much, much greater role for the Ukrainians than it does for the Russians.
6:22
That's true both because Ukraine had less equipment to begin with, and is also believed to have captured far more of it.
6:27
At time of recording, the Ukrainians are visually confirmed to have captured about 3,000 pieces of heavy equipment.
6:33
The number is about 1,000 going in the other direction. And over the course of the war, we've seen entire Ukrainian units
6:38
sometimes re-equipped with stuff of Russian origin. So whereas this was mostly an afterthought for our Russia analysis,
6:44
here it's going to be much more front and centre. Now before we start jumping into aggregate numbers around losses and resupply,
The Quality Question
6:50
there's another side of the coin I wanted to explore first: quality. When we talked about Russian losses and resupply 2 weeks ago this question was really important
6:58
because over time Russia was using more and more older equipment. We had to understand the potential difference that using a T-62 or D-20 howitzer might make
7:06
over the much more modern designs that dominated the pre-invasion Russian force. A one-to-one quantitative replacement in those cases
7:13
may not mean a one-to-one replacement in combat power. With Ukraine the question is also important, but sometimes runs in the other direction.
7:20
What's the impact of going from a Soviet howitzer to a PzH 2000, or from a BMP to a Bradley?
7:25
It's been said that quantity can have a quality all of its own, but quality can also have a quality all of its own.
7:31
The basic question then is to what extent does that actually matter, or is a tank a tank, and a gun a gun?
7:37
Intuitively in some cases the answer is obviously going to be "yes", and in others "not so much". The impact might be intuitive when the high quality piece of equipment
7:45
allows you to do something that is significantly different to what the lower quality piece enabled. In a war with very slow moving front lines for example, range is an obvious example here.
7:54
If you need to hit targets 60 kilometres behind the line for example, no amount of systems with 40 kilometre ranges are going to allow you to service that target.
8:02
Longer-ranged artillery might have a massive advantage in counter-battery engagements, and longer ranged anti-aircraft systems might actually be able to hit
8:09
aircraft using stand-off weapons like glide bombs. In other cases, the impact of a quality differential might be much more narrow.
8:15
All of that however is just based on anecdotes and what is intuitive. If you are watching this channel however, you are probably interested in what the data
8:22
(as imperfect and incomplete as it might be) may be able to tell us about some of the impacts equipment quality can have for Ukrainian forces.
8:29
If you watch interviews or talk to some of the Ukrainian personnel that use some of the better NATO equipment, one thing they'll often seem to say
8:36
is that it's better designed from a survivability perspective. Yes, your vehicle might still be knocked out by a mine, an ATGM or a loitering munition,
8:43
but even if the vehicle's done, the crew might survive to fight another day. When you are talking about modern IFVs or tanks, that sort of feedback
8:50
might make a degree of sense from a design perspective, as Soviet engineers were often just working with a different design philosophy
8:55
and different design priorities to their NATO counterparts. And while we don't really have data from both sides on how many crew on average
9:02
survive whenever a given vehicle type is hit, we can kind of hunt for clues using the visually-confirmed loss data.
9:08
The best clue for us here on how likely a crew was to survive comes down to what a vehicle looks like when it becomes a loss.
9:15
Differentiating between vehicles that have been thoroughly destroyed as compared to those that have merely been damaged and knocked out
9:21
is one of the most subjective and difficult things when it comes to operating these VC databases. It's also always possible that a vehicle was initially lightly damaged,
9:29
and then later thoroughly destroyed by for example a follow-up drone attack. But all else being equal, I think it's a fairly safe assumption to make
9:36
that more crew and passengers will survive a vehicle being knocked out and looking like this afterwards, something that would be classified as damaged, abandoned and captured in the database.
9:45
As compared to a destroyed vehicle like this one where the burnt out husk of the hull is in Ukraine, and the turret may be exploring a new career as an anti-satellite weapon.
9:53
And while it's generally possible for any vehicle to end up anywhere on that spectrum from lightly damaged all the way through to catastrophically destroyed,
10:00
the data we have suggests that in Ukraine they don't do so at regular rates. What you're looking at on screen there now is a chart
10:07
showing Ukrainian visually confirmed losses for 3 infantry fighting vehicles, the BMP-1, BMP-2 and the American Bradley.
10:15
I chose these three vehicles because the BMP-1 and BMP-2 are the most common infantry fighting vehicles in Ukrainian service,
10:22
with the BMP-2 obviously being the somewhat better vehicle. While the Bradley was selected because it's the only Western infantry fighting vehicle
10:28
where the number of visually-confirmed losses (which is approaching 100 at this point) gives us a sample size large enough for this exercise to potentially be worth doing.
10:36
The colour coding on the bar splits those losses into three types. Blue are those that appear to have been at least lightly damaged.
10:43
Orange are those that have been damaged and then either abandoned or captured. Abandoned vehicles may or may not be recovered by either side.
10:49
But the key point is the crew may have had more chance to survive. While the red represents those that are classified as destroyed.
10:56
Just in case that previous graph was a little hard to read because more BMP-1s have been lost than the other two types,
11:01
this is it expressed as a percentage of losses for each platform. Immediately you'll see something interesting, the distribution for the BMP-1 and BMP-2
11:09
(so more than 600 vehicles combined across those two categories) are pretty close. In both cases the blue bar is relatively small and the destroyed percentage is around 80%.
11:20
Meanwhile for the Bradley you see a lot less red and a lot more blue. And while the difference between blue and orange might matter if you're concerned
11:26
with whether or not the side permanently lost the vehicle as a result of the engagement, from a crew survivability perspective, it might be useful
11:33
to group them together as non-catastrophic losses. And that's exactly what you can see here with all those damaged,
11:39
abandoned and captured categories grouped together into the green bar, while the destroyed vehicles are marked in red.
11:44
I need to stress this is in no way a perfect proxy for crew survivability, but the pattern there is fairly stark.
11:50
If you are in a Ukrainian BMP-1 or BMP-2, and you become a visually-confirmed loss,
11:56
there's about an 80% chance that you are in a vehicle that is going to be thoroughly destroyed. If you are in a Bradley, then statistically the destroyed percentage drops below 50%.
12:05
That means that if this was a perfect proxy and everyone in a damaged vehicle tended to survive while everyone in a destroyed vehicle tended to not,
12:12
your odds of survival for those experienced IFV operators or the infantry carried would be more than 2.5 times higher in an American Bradley than it would be in a BMP-1 or 2.
12:23
And that's before we factor in things like the fact we often see infantry riding on the top of the BMP-1 or 2 rather than in the infantry compartment.
12:30
Which means you absolutely will see some drone footage where the infantry riding on top become casualties without the vehicle being knocked out.
12:37
The problem with trying to do this sort of analysis for other NATO vehicles that Ukraine has received is that often the sample sizes are just too low.
12:44
And that leaves the data vulnerable to being skewed by a range of factors that might have nothing to do with the basic survivability of the vehicle.
12:51
But because those factors can also be interesting, let me quickly tell you a story of three tanks. Over the course of the war Ukraine has received hundreds of tanks from its allies,
13:00
including relatively small numbers of some of NATO's best. These include the British Challenger 2, the American M1A1 Abrams,
13:07
and some later versions of the German Leopard, including the 2A6 from Germany itself, and the 122 from Sweden.
13:14
You could occupy internet experts for days at a time with debates over the comparative virtues of these vehicles.
13:20
To give a non-tanker's opinion, they are all capable, established MBT designs. But if you expect age to correspond with survivability,
13:27
you'd probably give the advantage to the German Leopard 2A6, as well as the Swedish version which is related to the earlier 2A5
13:33
with a major focus on vehicle and crew survivability. Then you might look at the visually confirmed loss data and quickly conclude
13:39
that those Leopard 2A6s and Strv 122s are obviously death traps. With 18 examples appearing in the loss data against about 31 supplied.
13:48
The loss rate for the M1A1 Abrams is much lower at around 30%. And the British Challenger 2 looks absolutely incredible,
13:56
with more than 90% of the fleet still undamaged. So does this mean we should all start yelling Rule Britannia,
14:03
and standardise on the Challenger 3 as NATO's new main battle tank? Well, before you get too excited and start trying to figure out where the Poms will fit
14:09
their new tank mega-factory, we can flip the graph into one categorising the type of loss. It's suddenly the Swedish Strv 122 which is by far the most survivable vehicle in our set.
14:20
While there are probably VBIEDs out there with better graphs than the Challenger 2, with 100% of Challenger 2 losses being catastrophically destroyed rather than damaged.
14:29
If you fed that figure to an AI with no other context, it might conclude that the Challenger 2 is a less survivable vehicle than the BMP-1.
14:36
So what's going on here? Issue one is that the bar says 100% because only one
14:42
Challenger 2 has been lost in Ukraine, and it looked like this. Problem two is that when only a few vehicles are involved rather than hundreds or thousands,
14:50
they tend to be concentrated in specific units and potentially used in different ways. Ukraine is believed to have received 14 Challenger 2s from the United Kingdom,
14:57
and they appear to have gone to the 82nd Air Assault Brigade. There have been a couple of interviews with tankers from the 82nd
15:03
who have described how they have generally used the Challenger 2 and how they evaluate it compared to other tanks.
15:09
Most them praise the combination of accuracy and protection on the Challenger 2, and described using it for relatively long-range engagements.
15:15
The one on the right there for example, describes engaging Russian defensive positions from ranges of between 2.6 and 2.8 kilometres.
15:22
That is a long way from maximum range for potential tank engagements, but it's also not exactly urban warfare.
15:28
Another interview I found talked about preferring the lighter T-80 for close-in assault work due to a combination of factors like the Challenger's weight on one hand
15:36
and its comparative accuracy on the other. Whatever the case may be, it sounds like you have the entire tank force
15:42
concentrated in a single veteran unit that, at least anecdotally, talks about using them in a relatively conservative fashion.
15:48
And there might be some threats that you are comparatively less likely to encounter if you're mostly doing long-range work with a vehicle, think anti-tank mines.
15:55
Meanwhile, a lot of the Leopard 2A6s found themselves being used as the spearhead of the Ukrainian offensive in June 2023.
16:02
They were largely handed over to mechanised units, and used for some of the most difficult jobs a unit could be assigned: breaching prepared enemy defences under fire.
16:10
In a sense you could argue this represented just another example of a phenomenon we've seen many times before on both the Russian and Ukrainian sides.
16:17
Some of the better available equipment being allocated to do the most dangerous and difficult of jobs. But without sufficient engineering assets to clear the minefields,
16:25
no ATACMS to suppress Russian attack helicopters, and all of the other factors that went into limiting the amount of progress that Ukrainian offensive made,
16:32
a fair number of these very expensive tanks ended up being knocked out or destroyed relatively quickly.
16:38
Finally the loss picture for the M1 Abrams has been different again, and can best be described as bumpy.
16:44
The vehicle avoided taking any losses during the breaching phase of the Ukrainian 2023 offensive, because it hadn't yet arrived.
16:50
And even once they did arrive, the losses we've seen have mostly been very clustered. 10 Abrams have so far appeared on the visually confirmed loss list, compared to 32 supplied.
16:59
But most those losses appear to have been clustered in a relatively short span of time. When it appears they may have been committed to blunt a Russian offensive.
17:07
The reason I bring this up is because a) it's interesting. And b) because I think it serves as a warning against jumping to conclusions based on incomplete data.
17:15
And to suggest we should probably think twice before replacing all the modern main battle tanks in NATO inventories with vehicles
17:21
the data suggests are obviously much more survivable, like you know, the T-55. You can't always blindly trust the data.
17:29
So where does all that leave us if we are trying to assess what impact quality might have on Ukrainian combat power going forward?
17:34
Without a whole bunch of additional information, the safest conclusion is probably that it matters, but we can't be sure how much it matters.
17:41
Although I imagine that for Ukrainian troops who end up surviving because they were in the back of a Bradley rather than a BMP,
17:46
the answer is already, at least at a personal level, all the difference in the world. But now it's probably time to start talking about the quantity side of the resupply and loss picture.
The Question Marks: Domestic & Captured
17:55
And in order to do that there's a couple of assumptions I need to advance up front. All of which relate to some of the biggest unknowns when it comes to equipment resupply.
18:03
Firstly, we generally don't have very good public estimates of how much equipment the Ukrainians themselves are producing or reactivating.
18:09
We know that budget is being allocated to it and that it is happening, but other than a couple of cases where I'll try and reverse engineer out
18:16
a rough estimate of Ukrainian production, for the most part most of the figures you are about to see assume the Ukrainian
18:22
defence industrial base has been on extended vacation since February 2022. Assumption two relates to how we count captured Russian equipment.
18:29
Especially in the earlier stages of the war, this may have been Ukraine's most significant source of heavy equipment resupply.
18:35
And with nearly 3,000 pieces of equipment marked as "captured" in the visually-confirmed loss databases at time of recording, we can't exactly ignore it.
18:43
At the same time, I think it would be massively over-optimistic to assume that every piece of equipment captured can be repaired,
18:49
put back into service and then maintained in service. So as in the past, when I am calculating inflows I am going to divide
18:55
the number of visually-confirmed captured vehicles by 3. Basically assuming that if you capture three T-80s for example,
19:01
you're probably going to be able to cobble together enough parts to operate at least one of them. Finally, there are often a whole range of ambiguities around
19:08
foreign military assistance and equipment deliveries. Some states provide military assistance, but never announce what equipment is in the packages.
19:15
Others might announce equipment but with ambiguous timelines, and it never being entirely clear what has or hasn't been delivered.
19:21
Unless otherwise noted, because we don't have any data to go on we are going to assume that secret deliveries don't exist.
19:27
We are also often going to be bundling together equipment that has been pledged with equipment that has already been delivered.
19:32
We'll break things out occasionally, but for the most part we are interested in the replacement pipeline, not just what's already arrived.
Losses
19:38
OK, so with that discussion around quality complete, let's get into the much more data-friendly issue of quantities.
19:44
And the way I thought we'd do this is start with the big picture around Ukrainian losses, then the question of inputs, and then put it all together
19:50
for a more detailed look at specific equipment categories. Of course until the AI apocalypse plays out, all of that equipment is going to need people to operate it.
19:58
So I think it's worth putting a note up front about the much harder topic to assess: manpower losses and replacements.
20:04
We can be fairly confident that Ukraine has taken significant casualties over the course of this war. And earlier this year I pointed to some of the evidence that serious manpower shortages were in play.
20:13
Actions to address that manpower shortage were arguably significantly delayed, although eventually we did see changes to Ukrainian mobilisation rules.
20:21
They finally passed in Ukraine a couple of months ago at this point, and reports indicate that induction rates have now increased significantly.
20:27
Any determination of whether or not these changes are likely to be enough to meet Ukraine's manpower requirements probably needs to wait a few more months.
20:34
But I will note that in amongst all this we have seen some interesting initiatives, including the formation of what has often been called a "Polish Legion".
20:41
The name there is arguably a little bit of a misnomer, because this is intended to be a volunteer unit formed of Ukrainian men currently living in Poland.
20:49
Basically what the Polish government has said, to paraphrase Foreign Minister Sikorski, is that many of these Ukrainian men do want to serve, do want to help rotate their compatriots.
20:57
But many have also expressed concerns about things like training standards, or their ability to eventually re-join their families.
21:03
So to answer those concerns, the Poles have set up a volunteer unit that will be trained by Polish forces first and then deployed to Ukraine as a cohesive unit.
21:11
The recruits won't be broken up to be sent as individual replacements to various other units of the Ukrainian military, they'll stick together and fight as a unit.
21:19
And reportedly within a couple of days of the initiative being announced several thousand had already registered.
21:24
I think it'll be interesting to see going forward both whether or not the Polish model works, and whether other European states adopt it. To quote the Polish Foreign Minister,
21:32
"If every European country did that, Ukraine would have several brigades." And one imagines that given these are volunteer units
21:39
that are meant to be properly trained before being deployed, they may have relatively good combat power compared to other green units.
21:45
But even if Ukraine is able to address some of its manpower shortages, materiel concerns are likely to remain acute.
21:51
The country began the war with less heavy equipment than the Russian Federation, significantly increased the size of its military without
21:58
a commensurate increase in the amount of heavy equipment available, and importantly for this part of the analysis, has also lost a lot of heavy equipment in nearly 2.5 years of fighting.
22:06
The first question then is: how much? What you're seeing on screen here are Ukrainian losses across
22:11
different equipment categories according to two very different sources. Those orange bars on the right are visually-confirmed losses, including vehicles
22:19
that were only damaged, which I think serves well as a kind of soft floor to the estimate. The red bars on the left represent an impossibly high upper-end estimate,
22:27
namely the claims of the Russian Ministry of Defence. And as you can see, there is a slight difference between them.
22:34
But I bring them up to illustrate a point around potential over-claiming. Both sides are likely to be over-claiming, that's pretty much
22:40
just the historical default for anyone fighting a war. But the data seems to suggest that the scale at which they are doing so is very different.
22:48
What you're seeing on screen there is a chart showing the relationship between the claims by both sides and the visual evidence for them.
22:54
That 15% for Russian aircraft claims for example, means that for every 100 Ukrainian aircraft that Russia
23:00
claims to have destroyed, we have evidence for about 15. On the other side, for every 100 Russian aircraft Ukraine claims
23:06
to have permanently grounded, we have evidence for about 33. The difference there is between over-claiming the evidence by about 3:1 and roughly 7:1.
23:14
For every 100 Russian rocket artillery systems the Ukrainians claim to destroy we have visual evidence for about 35.
23:20
An over-claim rate (or perhaps a better term here is actually "evidence ratio") of less than 3:1.
23:25
For the Russians, every 100 Ukrainian systems claimed will come with evidence of about 6, an evidence ratio of closer to 17:1.
23:35
Going forward, I'm mostly going to be using visually confirmed loss data for reasons I've discussed extensively before.
23:40
But I thought that discrepancy was interesting enough to be worth surfacing. Very generally speaking, Ukrainian claims tend to be likely high, but at least possible.
23:50
While if the Russian claims were true, the Ukrainians would have been defending the streets of their capital with thrown bricks and harsh language probably back in 2023.
23:57
What you're seeing on screen now then are the visually confirmed losses for both Russia and Ukraine across different equipment categories.
24:03
And I think the graph serves to highlight at least two observations. The first is that, leaving Russian losses aside for a moment,
24:09
this war has undoubtedly destroyed a massive amount of Ukrainian equipment. Nearly 1,000 infantry fighting vehicles, more than 860 tanks,
24:17
around 600 artillery systems and 500 APCs to name just a few. The second observation is that, while throughout this war
24:25
Russia has consistently lost heavy equipment at a much higher rate than the Ukrainians, the difference between the two isn't consistent across the categories.
24:33
You'll often hear that Russia has very generally had about 3 times the visually confirmed equipment losses as the Ukrainians over the course of the invasion.
24:40
But to an extent they haven't been losing the same stuff. For a lot of the real heavy-metal categories, think tanks, infantry fighting vehicles and AFVs,
24:48
the ratio is often closer to 4:1, for multiple rocket launchers around 7:1. And in the command and communication category nearly 15:1, 283 to 19.
25:00
But there are also categories, quite numerically significant categories, where Ukraine has been losing more vehicles than the Russians have.
25:07
The thing is, these tend to generally be comparatively lighter categories of equipment that NATO has been able to supply in large numbers.
25:14
For example, the Ukrainians have lost more than 480 infantry mobility vehicles (think things like Humvees) compared to the Russian 265, a ratio of 1.8:1.
25:23
The ratio for MRAPs, wheeled mine-resistant vehicles (think things like the MaxxPro or the Bushmaster) is closer to 4:1.
25:30
On one hand, as we'll discuss later, this might signal that Ukraine is comparatively short on heavy equipment and has to use more light stuff instead.
25:37
On the other hand, it also suggests that wider 3:1 ratio across all categories might not truly represent the scale of the difference we've seen so far.
25:45
If you are trying to win a long-term war of attrition, you probably don't want to be trading tanks and infantry fighting vehicles for Humvees and M113s.
25:54
But whatever the disparity between the two sides, the fact is, especially once you account for non-visually confirmed or non-combat losses,
26:01
those graphs show a huge amount of Ukrainian equipment that needed replacement. This brings us to the question of inputs that Ukraine has turned to to balance out their losses.
Inputs
26:09
Overall, the most important source has almost certainly been deliveries from abroad. At this point dozens of nations have supported Ukraine with various financial and military resources.
26:18
Some have done so very loudly, giving us specific numbers and delivery timelines whenever new aid is forthcoming.
26:24
Others like Finland have been forthright about the fact that aid is being sent, but not about what's in each potentially explosive shipment.
26:31
And then somewhere in between there are those that appear to have tried to be transparent, but sometimes seem to have a little bit of trouble with the accounting.
26:38
With the US recently announcing that they'd found another roughly 2 billion US dollar over-count in the amount of equipment that has been sent to Ukraine,
26:45
because once again equipment appears to been valued based on how much it would cost to replace it with new equipment,
26:50
as opposed to how much the old stuff was actually worth. Understanding those ambiguities as best we can, the chart on screen
26:57
shows some rough estimates for how much equipment across various equipment categories is estimated to have been pledged or delivered to Ukraine from foreign sources.
27:05
As you can see, Ukraine has been receiving serious numbers of some lighter systems (like MRAPs, APCs) and infantry mobility vehicles (things like Humvees).
27:13
But also a fair amount of metal on those heavier line items. Overall the pledges amount to about 900 tanks, more than 1,000 infantry fighting vehicles,
27:21
3,500 armoured personnel carriers, and more than 5,000 infantry mobility vehicles.
27:26
Including 3,000+ Humvees. That might sound like a lot of Humvees, but we have to remember the US reportedly left the Taliban with about 12,000 of them.
27:34
The other key input we are concerned with capturing today is equipment the Ukrainians have managed to get from the Russians.
27:40
Even before the first heavy equipment deliveries began from NATO states, the tractor brigades were at it in Ukraine, and a significant
27:47
number of Russian vehicles found their colours turned. At time of recording, the estimated more than 2,900 pieces of captured equipment
27:54
include more than 500 tanks, 271 AFVs, 616 IFVs,
28:00
and an embarrassingly large number of command vehicles. But while I think it's important to model the fact that this mode of equipment acquisition
28:08
was no doubt significant for the Ukrainians, those figures don't include the divide by 3 adjustment that I would generally suggest.
28:14
And I think the data suggests that significant aid packages from the Russians may be less common going forward.
28:20
What you can see on screen here is the change in the number of tanks, IFVs and AFVs captured by the Ukrainians across three distinct periods.
28:27
What you can see on screen here are massive numbers for captured vehicles during that February to May 2022 period,
28:33
even larger numbers between May '22 and May 2023, and then low, or in some cases even negative numbers for May 2023 through '24.
28:43
I think the most likely explanation here is that these three time periods reflect very different stages of the war and different major events.
28:49
February to May 2022 is the era of the Kyiv Goodwill Gesture, where the Russians apparently demonstrated the scale of their goodwill
28:56
by leaving hundreds of armoured vehicles behind as they pulled out of Ukraine's north. The year between May 2022 and May 2023
29:03
includes such minor events as the Kharkiv Counter-Offensive. That saw Russian defence lines collapse, a number of major bases overrun,
29:11
with huge amounts of equipment left behind or vacuumed up in the process. Both of those represent periods of relatively rapid movement.
29:18
A time when vehicles might be left behind or overrun, and thus more likely to be captured. By contrast, that final 12 month period has been
29:26
overwhelmingly dominated by relatively static fighting. Vehicles being lost might be lost in the grey zone
29:32
or behind enemy lines, making them very difficult to recover. And during that period they were also probably more likely to be subjected
29:38
to follow-up attack by drop drones or FPVs, which we've often seen used to completely destroy vehicles in order to prevent them being recovered.
29:46
The collapse in the rate of equipment captured is so great, that with tanks you actually see a negative number, meaning the list got smaller over those 12 months.
29:54
This reflects the fact that over time you are constantly seeing efforts to try and find and remove duplicate entries from the databases.
30:00
And that where visual evidence emerges of what was presumed to be a captured vehicle being destroyed, for example by a follow-up FPV strike,
30:07
that vehicle will then if possible be reclassified from captured to destroyed. Normally that process is all but invisible in the data
30:14
because new losses being reported tend to be more than enough to compensate. But as capture rates dived over those 12 months, those corrections and changes became very visible.
30:23
The key takeaway is that Ukraine is probably even more dependent now on domestic production and allied deliveries than it was in 2022 or 2023.
30:31
The Russian military may have fronted the Ukrainians a "Build Your Own Mechanised Force" starter pack or two,
30:36
but it's hard to imagine capture rates climbing to those sort of levels again unless the lines become much more dynamic.
30:41
But now with some of that big picture around equipment losses and inflows in place, I want to start weaving all those items together and try to better understand
30:49
or at least guesstimate, how things might be netting out for different parts of the Ukrainian force. So to jump into some specific equipment categories,
MBTs
30:56
if we are leaving artillery and air defence systems for a future episode I figured we could go for something of a heavy metal opening. So let's talk tanks.
31:03
At a big picture level, tanks have had a rather interesting war in Ukraine so far, especially from a PR perspective.
31:08
There was the early panic over whether this war signalled the end of the tank. And in 2024, we've seen time and time again that heavy armour
31:15
can be vulnerable to a range of systems including landmines, ATGMs and drones. I've heard testimony from people who have fought in Ukraine
31:22
that they as infantry don't want to be anywhere near the friendly tanks, because the sheer amount of shit they tend to attract.
31:28
But despite those drawbacks, both sides also continue to swear by them. We've seen them used defensively to stage armoured ambushes, offensively to lead assaults,
31:37
deployed as essentially very heavily armoured artillery systems, or used to pick apart defensive positions, fortifications and structures
31:44
before infantry move in for assault actions. In other words tanks in Ukraine have proven to be vulnerable,
31:50
but then again, so too is basically everything. And the combination of a big gun and hopefully decent sensors
31:56
on a mobile and well protected platform still seems to be something that the armies can appreciate. While there have obviously been some peaks and troughs,
32:03
overall Ukrainian tank loss figures show a degree of stability. What you can see on screen there are the approximate visually-confirmed losses
32:09
for the period May to June 2022, May to June 2023, and May to June 2024.
32:15
I've chosen these time periods to try and give a somewhat like-to-like comparison between the 3 years, while leaving aside the somewhat dramatic outlier
32:22
that was the very earliest stages of the invasion. What you can arguably see is that while there is an up-tick in the middle there,
32:28
which has probably corresponded with the beginning of Ukraine's 2023 offensive, the burn rate between the three periods doesn't appear to have swung that wildly.
32:36
What that might suggest to us (even though it's clearly not enough information in and of itself) is that unless the loss rates of the tanks that are active have been changing significantly over time,
32:46
the total size of the fleet has probably remained relatively stable. That would broadly align with a February assessment by the IISS that at the time of that report's writing,
32:55
"We estimate that 2 years after the full-scale invasion, the number of MBTs in service with the Ukrainian armed forces remains near pre-war levels."
33:03
If that's true, the whole demilitarisation thing is clearly going super well. As we saw from those loss charts earlier, a stable Ukrainian MBT inventory
33:11
can't be because they're not losing tanks, they are, and they've lost a lot of them. Roughly 870 destroyed, damaged or captured as at time of recording.
33:20
But against those estimated losses there have also been significant inflows. About 900 main battle tanks have been pledged to Ukraine by its various allies,
33:28
of which about 2/3 are believed to have been delivered. For the most part, you can think of those pledges as falling into three very broad waves.
33:36
There was a massive initial surge of Soviet and post-Soviet designs that began quite early in 2022.
33:42
Then a push leading up to the 2023 offensive to get Ukraine at least some modern Western main battle tanks to augment its force.
33:48
And then what appears to basically be a numbers-making exercise to get Ukraine some older NATO tanks, specifically the Leopard 1.
33:55
Although a vast majority of those are being reactivated from storage and so haven't made it to the battlefield yet.
34:00
I also note there's sometimes a tendency in coverage to dismiss the importance of smaller announcements that we do see from time to time,
34:06
such as the recent report that The Netherlands and Denmark will be sending another 14 Leopard 2A4s to Ukraine this summer.
34:12
We are now at the point that over 100 Leopards 2s, or related designs like the Strv 122, have been pledged or delivered to Ukraine.
34:19
And for the most part that figure has been built up one small announcement at a time. While there have been a huge array of countries involved
34:25
in trying to get Ukraine more battle tanks, there are a couple of efforts that appear to have really moved the needle from a supply perspective.
34:31
The largest single line item for allied tank supply to Ukraine probably remains Poland's 250+ T-72s and 60 PT-91s.
34:40
The problem from the Ukrainian perspective is that, much like Russia's donations in 2022 and 2023,
34:45
it seems very unlikely we'll be seeing those sort of donated numbers from Poland again in the near future, as the country is now left waiting for deliveries of new main battle tanks such as its K2s from Korea.
34:55
The most common single type of NATO tank to be delivered by contrast, is likely to be the Leopard 1 with about 155 either in the pipeline or already delivered.
35:04
And of course we can't forget Moscow's generous contributions that we discussed earlier, although as per our stated assumptions,
35:10
I'm only going to assume about a third of them can be put into action. Put that all together and you get the chart you see here.
35:16
What I've done is broken up the visually confirmed losses and potential inflows, things like captured vehicles, domestic production or foreign support,
35:23
and divided those into Soviet, post-Soviet and related vehicles on the top and NATO-type vehicles on the bottom.
35:28
In part based on the data work we did back in the Russian losses episode, for illustrative purposes I very roughly estimated the number of Ukrainian tanks
35:35
reactivated from storage at about 200 since February 2022. Net that all together and you end up with Ukraine being up by about 470 tanks overall,
35:44
270 if you leave out the reactivation component. Adjust down for approximately 300 of the pledged tanks
35:51
being shown here not having been delivered yet, and you end up with the Ukrainian fleet being slightly negative if you don't account for reactivation, to slightly positive if you do.
35:59
In other words, you are at the sort of point where potential errors in every direction, be it losses that don't become visually confirmed,
36:05
or damaged vehicles that end up being repaired rather than written off, could fairly easily move the result either side of zero.
36:12
To me, what that suggests (and we could very easily be wrong) is that after nearly 2.5 years of hard attritional fighting
36:19
the active Ukrainian tank fleet is probably pretty close to where it began. With the biggest changes being the shift in the composition of the force, not the size of it.
IFVS & APCS
36:27
Moving on, let's look at a core component of many mechanised forces in Ukraine, the infantry fighting vehicle.
36:33
Perhaps even more so than tanks, infantry fighting vehicles (especially the old BMP series) have been absolutely ubiquitous in both Ukrainian and Russian service.
36:41
They are intended to combine the ability to transport (or attempt to transport) infantry safely across a very dangerous battlefield
36:47
and pack a useful amount of firepower, often in the form of an auto-cannon and/or missile armament. By comparison, armoured personnel carriers like the American M113
36:56
would normally be expected to lean much more into the sort of battle taxi role. Providing some protection to the troops they carry,
37:02
but often having a significantly lesser armament. Both types of vehicles can serve to protect infantry from things like shrapnel.
37:08
But you probably don't want to be using them in the same sort of way that you would use the more heavily armed IFVs.
37:13
As you can see on screen there, Ukraine's loss patterns for the two vehicle types have changed in different ways over time. In both cases Ukrainian losses
37:21
in May to June 2023 leading into the Ukrainian counter offensive were significantly higher than in May to June 2022.
37:27
But the rate of increase was significantly different, APC losses more than tripled, while IFV losses increased by about 75%.
37:34
Then in May to June 2024 with Ukraine now back on the defensive, IFV losses would drop back a bit while APC losses would continue to grow.
37:42
Adjusting for the very different state of the war across those three time periods, what this would seem to suggest is that the Ukrainian IFV fleet
37:49
has probably been broadly stable between 2023 and 2024, assuming you are likely to lose fewer vehicles on the defence than the attack.
37:57
While all else being equal, you'd probably guess that the APC fleet has grown. And to test that idea a bit, we can start to pick apart
38:03
the visually confirmed losses and the estimated resupply. As with the tanks, what I've done on screen here is split up the visually-confirmed losses,
38:10
announced pledges or purchases, and an estimate of vehicles captured, into the Soviet/Russian/Ukrainian category on the top
38:17
and the NATO vehicle category on the bottom. What it shows is Ukraine falling short on replacing its losses
38:23
of Soviet infantry fighting vehicles while running a surplus on NATO ones. Ukraine is visually confirmed to have lost about 830 Soviet IFVs,
38:30
for an inflow of about 670. For NATO infantry fighting vehicles it's a loss of 137 against an inflow of 729.
38:39
I'd also note that as you see on screen there, the Ukrainian BMP fleet at least on paper, is only avoiding being in an even worse state,
38:45
because of an inflow of an estimated 205 recovered captured vehicles. A number which is only possible because Ukraine is visually confirmed
38:53
to have captured a staggering 616 from the Russians at time of recording. If you net those figures for Soviet and NATO IFVs together,
39:01
Ukraine ends up positive by about 430 vehicles. But if you then adjust down for the fact that potentially hundreds of these vehicles
39:08
may not have been delivered yet, you end up with a much more humble increase. More or less what the loss data appeared to be hinting at a moment ago.
39:15
While there were a lot of estimates involved in that calculation, I think what is abundantly clear is that whatever the size of the IFV inventory,
39:22
the composition of that inventory in Ukraine is absolutely changing over time. Replacements may not be enough to keep up with BMP losses,
39:29
while pledges of NATO infantry fighting vehicles have so far outstripped losses. There are several NATO IFV designs we have seen become more common over time.
39:37
These include the German Marder, of which Ukraine has already received more than 100 and will eventually receive 140 in total I believe.
39:44
The US-made Bradley has been even more significant and fought a number of high-profile engagements. Ukraine is believed to have received just over 300 Bradleys at this point,
39:53
and there's a question over how many more might be received in the future. The Swedish CV90 has appeared in much smaller numbers,
39:59
but I bring it up here because this is the vehicle with potentially the longest tail. With both the Bradleys and the Marders, generally what Ukraine expects to receive
40:06
is older reserved or reactivated stock. With the CV90 we've actually seen plans for some
40:11
new-build production that might sustain the force into the future. In that respect you might describe the CV90 as being for the Ukrainians
40:18
what BMP-3 is for the Russians, the modern option that isn't available in anywhere near sufficient numbers now,
40:23
but which will continue to trickle into the force long term because it's the one in active production. Although it has to be said there's a wide gap between the number
40:31
of BMP-3s that Russia is believed to be producing and the rate of anticipated CV90 production for Ukraine.
40:37
Before we move on, I do want to flag one potential anomaly that does come through in the IFV data. For most types of IFVs in Ukraine there tends to be a relatively intuitive relationship
40:46
between visually confirmed losses and the number of vehicles supplied. Three times as many Bradleys have been sent as Marders,
40:52
and Bradley losses are about 3.5 times Marder losses. The anomaly though comes in when we are talking about the Polish Wolverine IFV.
40:59
It was announced back in April 2023 that Ukraine would be getting 200 of these things, including 100 in the very short term.
41:06
We then did start to see images of the vehicles in Ukraine being apparently operated by Ukrainian troops.
41:12
So by those accounts the Wolverine should be the second most numerous NATO IFV in Ukrainian service, second only to the American Bradley.
41:19
But while we've seen 93 Bradley losses, 20 Marders and 6 CV90s, the total visually confirmed loss figure for the Wolverine is 4.
41:28
Unless the Poles have secretly found a way to manufacture IFVs out of adamantium, I'm not sure I have a ready answer to this quirk in the data.
41:35
There are any number of things that could be driving it, deliveries could be less than anticipated, the vehicles may have been allocated to less dangerous fronts,
41:42
been very lucky and not destroyed as often, or when destroyed not photographed as often. It may also simply have just been used in a very different way by the Ukrainians,
41:51
because while it does have an IFV-type armament, the Wolverine, unlike the Bradley, Marder, CV90 and BMPs, is a wheel vehicle not a tracked one.
41:59
But in the absence of a definitive answer, for now I'll just leave this mystery to the audience. While Ukrainian IFV losses and resupply figures have,
42:07
at least on paper, remained relatively closely balanced, those same figures for armoured personnel carriers very much haven't.
42:13
As you can see there, the picture for Soviet, Russian and Ukrainian designs, mostly things like BTRs, has remained relatively balanced.
42:20
But the visually confirmed loss of fewer than 300 NATO APCs has so far been balanced by announced inflows of more than 3,300.
42:29
Producing a total net on paper position of positive 3,060 vehicles.
42:35
Accounting for vehicles that may have been pledged but not yet delivered lowers that figure to around positive 1,700.
42:41
But hopefully we can all agree that even by the standards of this war, that's still a lot of vehicles.
42:46
And a lot of that figure has been made up by various versions of an old Cold War stalwart, the American-designed M113.
42:53
In terms of NATO APCs, the M113 is about as simple as it gets. In its basic version the thing is basically a metal box
43:01
to which you add some armour plate, an engine, and some tracks. Troops can then hide inside said metal box while it drives around,
43:08
because marching long distances in a heavy pack sucks, and so too does shrapnel. As to which of those two things sucks more,
43:14
that will probably come down to the individual infantryman you talk to. If you go through public Ukrainian reviews of the M113,
43:21
you'll find things they generally like, like its simplicity, and some things that are viewed less favourably.
43:26
In an ideal world this probably wouldn't be the vehicle they'd be getting, with a focus instead on more IFVs or modern APCs.
43:32
But as I previously indicated I think as far back as 2022, if Ukraine's allies were ever going to try and supply anywhere close
43:39
to the sheer number of armoured vehicles that the country likely needs, and even more try and do it without providing 30 distinct designs
43:46
thus extending the torture already experienced by Ukrainian maintainers, the M113 was always going to be one of the only potential answers.
43:53
Because basically everyone in NATO has them, they have a lot of them, and also basically no one wants them.
44:00
As you can see on screen there, Ukraine is believed to have received M113s from The Netherlands, Lithuania, Australia, Denmark, Germany, Poland, Spain,
44:07
the United Kingdom, Italy, Belgium, Luxembourg, and a casual 900 (including 300 medical vehicles) from the United States of America.
44:16
That's a lot of metal boxes, but to give a sense of perspective, it's still fewer than the more than 1,000 vehicles the US supplied to the Iraqis.
44:23
M113 is not a particularly modern, well armed or well protected vehicle. But you can potentially fit dozens or hundreds of the things into an aid package
44:32
that might otherwise pay for a handful of tanks, or maybe one or two air defence systems. And so we should probably expect to see more and more of them appearing in Ukraine as time goes on.
44:42
Not just as basic APCs, but also as an array of specialised vehicles. Especially since, as we'll come back to, you still see plenty of cases
44:50
of Ukrainian units having to use unarmoured civilian vehicles to do things like evacuate wounded when a dedicated armoured option would probably be preferable.
44:58
Ukraine likely has far more APCs than it started with, but they probably also wouldn't say no to a couple of thousand more.
Aircraft
45:04
While the Ukrainian ground forces may be treading water in some equipment categories and potentially building up in others,
45:10
I think the data suggests that the Ukrainian Air Force is in a much more difficult position. If you just look at the headline picture, you might get the impression things are relatively neutral.
45:19
97 visually-confirmed aircraft losses at time of recording, up against slightly more than 130 aircraft delivered or pledged as resupply.
45:27
On the surface that might seem to suggest that the Ukrainian Air Force is in better condition now than when Russia invaded in February 2022.
45:34
But for a variety of reasons I'd suggest that's probably not the case. The Ukrainian Air Force has been flying hard and constantly for almost 2.5 years at this point.
45:42
Constantly displacing between different airfields trying to evade Russian long-range strikes. As a result that 97 figure should probably be considered a floor,
45:50
as other aircraft have likely been worn out or lost due to other factors. And likewise that 128 figure probably doesn't include some reactivated airframes.
45:58
In particular, things like Sukhoi 24s. But the two most significant asterisks I think are a) delivery timelines that we'll talk about in a moment.
46:06
And that secondly, losses and resupply aren't impacting different parts of the force evenly. On paper for example, Ukraine's force of its most numerous fighter aircraft,
46:14
the MiG-29, seems to basically be treading water. 30 MiG-29s have been visually-confirmed destroyed or damaged since February 2022,
46:22
while 27 MiG 29s are believed to have been delivered by Poland and Slovakia, although some of those were likely intended as parts donors.
46:29
In recent weeks Poland has raised the prospect of potentially sending another squadron of these aircraft to Ukraine, but likely only after it receives new aircraft to replace them.
46:38
That pushes any potential timeline well out, so I haven't included them here. One of the many challenges for the Ukrainian MiG-29 fleet
46:45
is that while replacements haven't quite covered visually-confirmed losses, the number of tasks being assigned to the fleet has likely increased over time.
46:52
We've seen the aircraft used to do things like hunt Russian drones, or deploy HARM anti-radiation missiles against Russian air defence systems.
46:59
And for a force that was estimated to have fewer than 40 active MiG-29 airframes at the time of the Russian invasion,
47:05
even just those tasks alone would likely be a significant ask for the force, given the length of the Ukrainian front line.
47:10
But as hard as things appear to have been for Ukraine's MiGs, the state of its Sukhoi 24 and 25 squadrons is likely even harder.
47:18
Pre-invasion, the Sukhoi 25 was an important ground attack aircraft for both the Ukrainians and the Russians.
47:23
Ukraine was estimated to have just over 30 active airframes of the type. And there are plenty of videos of Ukrainian pilots out there flying the type
47:30
at altitudes so low that you might think it's an air show manoeuvre. If not for the flashing lights on the radar warning system
47:36
and the fact the pilots appear to be firing off volleys of rockets. As a rule, the use of live ammunition is generally discouraged during air shows.
47:44
Remarkably, flying right up to the front line and then attacking with relatively short-range weapon systems appears to have been a highly dangerous exercise.
47:51
And Ukraine has suffered the visually-confirmed loss of 20 of the type since February 2022,
47:56
against maybe 4 pledged replacements from abroad. In proportional terms, the visually confirmed losses have been even higher
48:03
for Ukraine's fleet of Sukhoi 24 Fencers. These swing-wing strike aircraft appear to have been viciously hunted by the Russians
48:09
because they are the ones capable of carrying Storm Shadow missiles. A weapon which has caused a number of very high-profile inconveniences
48:16
for Russian forces, including the addition of some nice new ventilation holes to at least one Russian submarine.
48:21
19 Sukhoi 24s appear in the visually-confirmed loss database, and the only replacements are believed to be the reactivation of stored Ukrainian models.
48:30
In other words as best we can tell, Ukraine's fixed-wing attack aviation has been worn down over time to a fraction of its pre-war strength.
48:38
At least in raw quantitative terms, the offset for a lot of these losses is meant to be the American F-16.
48:44
From relatively humble beginnings, the number of pledged F-16 airframes has steadily climbed. The figure has reached roughly 95 at time of recording,
48:52
which, if they all arrived tomorrow, would be enough to replace every aircraft Ukraine is visually confirmed to have lost since February 2022.
48:59
However, as soon as you start to dive into the fine print that 95 figure starts to come down significantly.
49:04
Some of the airframes pledged for example, are only ever intended to be parts donors, not operational aircraft.
49:10
Another major issue is timelines. Belgium has pledged about 30 airframes with the first to be delivered this year, but the final ones may not arrive until the end of 2027.
49:19
Meaning the final deliveries are hopefully much more relevant to Ukraine's post-war security position than the current invasion.
49:26
There's a couple of times on this channel where we've discussed the debate around how much of a game changer F-16 is likely to be,
49:31
and how its capabilities measure up against the various Soviet and Russian aircraft in play. But wherever you stand on that particular debate, I think there's one area
49:39
where the potential impact of this platform in Ukraine is very clear. Whether you think F-16 is a bad plane, a good plane,
49:45
or god's gift to fighter pilots, the fact is it is a plane. And if it wasn't for the effort to train Ukrainian pilots and ground crews,
49:52
source these airframes and eventually get them to Ukraine, then the medium-term projection for the Ukrainian Air Force would probably be
49:59
that the Russians would succeed in grinding it down to combat ineffectiveness. Realistically for Ukraine at this point it isn't a choice between a fleet of F-16s,
50:07
or a fleet of Gripens, or a fleet of F-18s, or a fleet of F-35s (as hilarious as that would be),
50:12
it's a choice between an F-16 fleet and not having a fleet. Russia has spent almost 2.5 years putting fewer
50:19
than 100 Ukrainian aircraft on the visually-confirmed loss list. F-16 has the potential to replace a lot of them,
50:25
and with a platform compatible with a wider range of NATO weapon systems. Ukrainian pilots may not have a wonder weapon, but they will have a weapon.
50:33
While the Russians face the prospect of basically having to start the process of destroying the entire Ukrainian Air Force all over again.
50:40
The Russians have had a number of recent successes flying reconnaissance drones deep into Ukrainian territory and then launching pinpoint strikes with systems like Iskander.
50:48
But only time will tell whether the tempo of those strikes increases, decreases or remains about the same.
50:53
In terms of the combat power of the Ukrainian Air Force out into 2025, I think there are three other elements we need to cover.
50:59
The first is the arrival of potential enablers. The kind of systems that may not be the most numerous,
51:04
but might help Ukraine get the best out of the aircraft it has. The biggest example I can think of here is that Sweden has pledged
51:10
to provide Ukraine with two light AWACS aircraft. These won't have the same capabilities as the much larger AWACS aircraft flown by forces like the USAF,
51:19
but because they'll be under Ukrainian control they will be able to fly in Ukrainian airspace. And if they are deployed carefully, they might be fairly useful to Ukrainian defensive efforts.
51:28
The second big question is whether some long-hypothesised transfers will actually eventuate. We are talking for example about Mirage 2000s from France.
51:36
We've been told that some will eventually be coming, but not how many or how soon. There's also been some discussion and speculation
51:42
around Ukraine potentially sourcing some FA-18 Hornets. And reportedly more advanced discussions around Ukraine potentially receiving
51:49
an aircraft almost custom designed for the war they are now fighting, the Swedish Gripen. It was first reported that Ukrainian pilots were testing and training on the type back in 2023.
51:59
But so far we haven't heard any announcement that the aircraft will actually be delivered. Of all the fighters we've just discussed, airframe for airframe,
52:06
the Gripen would arguably be the most useful. But in the absence of any announcement, it doesn't make it into our data.
52:12
The final question has very little to do with airframes and everything to do with munitions. All else being equal, the combat value of a fighter or multi-role aircraft
52:21
can vary wildly based on what's on the rails. Ukraine's MiG-29s and especially its Sukhoi 27s can have impressive performance characteristics,
52:29
and compared to many NATO aircraft they do. But they're also mostly stuck using a diminishing supply
52:35
of old Soviet air-to-air and air-to-ground munitions. With the arrival of F-16s, the compatibility should be there
52:41
to unlock much more of NATO's air-delivered arsenal. But we haven't heard much in the way of public announcements
52:46
to supply any of those potential weapon systems. An F-16 equipped with the AGM-154 Joint Stand-off Weapon
52:53
is going to be able to engage targets much further away than one using JDAM. And if the unlikely decision was ever made to supply them,
52:59
an F-16 equipped with the longest range versions of the AGM-158 could potentially target airfields or industrial targets a casual 500+ miles into Russian territory.
53:10
There would be nowhere in the Black Sea that Russian warships and submarines would be out of range. Nor would there be any Russian fighter air base
53:17
within a useful distance of Ukraine that couldn't potentially be targeted. That's very much an extreme scenario, and given the demonstrated escalation appetite
53:24
of the US government to date, a very, very unlikely one. But I think it's a reminder of how important it can be to talk ordnance, not just airframes.
53:32
In terms of a very rough, uncertain overall picture, the Ukrainian Air Force started the war with a much deeper equipment deficit
53:39
relative to its Russian opponents than did the Ukrainian ground forces. The technological and quantitative gap was much wider,
53:46
and Russia has continued to make Ukrainian air power a primary target. Between advanced Russian air-to-air missiles, air defence systems and attacks on Ukrainian air bases,
53:55
a significant port portion of Ukraine's pre-war airframe inventory has probably been worn down at this point.
54:01
Some like the MiG-29s have been broadly sustained by an infusion of new parts and airframes. While at this point the Fencer fleet, which continues to occasionally launch Storm Shadows,
54:09
probably only exists in any sense at this point because of repaired, reactivated or re-purposed airframes.
54:15
But if nothing changed, the overall trend was unambiguously towards Ukraine running out of airframes over time.
54:22
That's why the timing and scale of F-16 deliveries and any other Western jets that might eventually be announced are so important.
54:28
The Ukrainian Air Force is arguably currently in a period of great vulnerability. It's lost 2.5 year's worth of airframes, but it hasn't really received its next generation yet.
54:38
And while, yes, we should expect to see F-16s shot down or destroyed in Ukraine just as the MiG-29s and Sukhoi 27s have been before them, the fact is they will be there,
54:47
giving Ukrainian pilots something to fly and Ukrainian commanders options. But as to when we'll start seeing that transition happen in a big way, only time will tell.
Changing Force Composition
54:56
So where does that arguably leave us then in terms of some big picture observations around Ukrainian equipment losses and resupply as we go further into the second half of 2024?
55:05
Just as with the Russian military, I think there is plenty of evidence that the Ukrainian military's equipment profile is changing.
55:11
The difference is that while the age and quality of a lot of Russian equipment is going backwards over time, in Ukraine the direction of travel on quality tends to be
55:18
much more positive, although it is still a mixed bag. If you are replacing Soviet-era tanks with a Leopard 2A6 you can confidently say that's an upgrade.
55:26
But change that Leopard 2 out for a Leopard 1 and it's a very different picture. If you want to move from talking quality to quantity,
55:33
I think there are reasons for both Kyiv and Moscow to be deeply frustrated. In terms of some of the key heavy equipment categories, like tanks and infantry fighting vehicles,
55:41
as we showed, Ukrainian forces are likely roughly treading water. There may have been some decline or growth in the scale of the force,
55:48
but generally speaking replacements and losses seem to be relatively evenly matched. And frankly that's probably not the result either side is looking for.
55:56
For Russia it means that after almost 2.5 years of full-scale fighting, the expenditure of vast amounts of men and materiel
56:03
and deep withdrawals from the old Soviet stockpiles, their invasion to "demilitarise" the Ukrainian military is still very much at the starting line.
56:11
Or in some equipment categories has walked back a couple of metres and back into the stands. They also appear to be in the process of prosecuting a war they claim
56:19
to partly be about keeping Ukraine away from NATO, and in the process driving the transformation of the Ukrainian military
56:25
from an essentially post-Soviet force into a much more NATO compatible one. If the Russian intent was to eventually win through grinding attrition,
56:33
from an equipment perspective they still have a very, very long way to go. On the other hand from the Ukrainian perspective, supply matching losses
56:41
is likely to be viewed as nowhere near sufficient. Even if the Ukrainian tank and IFV forces were exactly the same size
56:47
as they were in February 2022, the Ukrainian army isn't. The country has mobilised, many new brigades have been established
56:54
and on paper they should get equipment too. But especially when you are talking about some of the heavier and rarer stuff
57:00
(think infantry fighting vehicles, tanks or air defence systems) the evidence suggests that there just isn't enough to go around.
57:06
We've seen reporting that some of the new brigades Ukraine intends to form are going to be infantry formations, not armoured or mechanised.
57:12
And Ukrainian Commander Syrskyi himself has reflected on the shortage of things like MANPADS and short-range air defence missiles.
57:18
It's an interesting flip side to the general estimation that Russia is using more armour in Ukraine now than it was in February and March 2022.
57:25
Russia has been able to dial up the number of tanks and armoured vehicles deployed in Ukraine through the relatively simple expedient of deploying
57:31
a greater and greater percentage of its military there, as well as generating new units. Even if the equipment those generated or regenerated units
57:38
are being issued is getting older and older. In general I think when it comes to things like tanks, IFVs, and certain other equipment categories,
57:45
the best characterisation is that Ukraine has been given enough equipment to sustain it in the fight and continue to hold back and grind down Russian forces.
57:52
But it hasn't been given enough to enable it to build up its own strength and throw some more decisive blows in return.
57:58
And I think while the data suggests that might be the case, I also think it's consistent with our broader observations about the war.
58:04
After all, whenever a statistical model and reality disagree, reality wins, and the model has to go back to the drawing board.
58:10
In this case our data suggests that Ukraine started with fewer tanks, IFVs or aircraft than Russia,
58:15
has received considerably fewer replacements than Russia that has been able to pull things out of storage by the thousands.
58:21
But at the same time has consistently been able to inflict disproportionate equipment losses. That seems consistent with the kind of grinding, slow moving, attritional war that we're seeing in 2024.
58:31
Whereas if Russia had been inflicting more losses than the Ukrainians, or the Ukrainians had been receiving far more resupply,
58:37
we probably would have expected to see the momentum of the fight shift more decisively in one direction or the other.
58:43
There are a number of silver linings in Ukraine's equipment supply picture. Domestic production has reportedly been increasing,
58:49
but largely hasn't been included in this analysis. Some high quality systems have been provided, and there are some equipment categories,
58:55
particularly light vehicles, where resupply volumes have far exceeded visually-confirmed losses.
59:00
Even if pulling together tank and IFV packages has occasionally been shaky, the MRAPs, IMVs and M113s have continued to flow.
59:08
The unfortunate side of that however is that it means in some cases Ukrainian forces might be forced to use light vehicles
59:14
when ideally a heavier option would be preferable. We have plenty of evidence of Ukrainian units using light elements
59:20
like MRAPs or Humvees in offensive actions not necessarily because they are always the right vehicle for the job,
59:26
but because sometimes you just have to make do with what you've got. These systems are still obviously very useful for Ukraine in a variety of scenarios,
59:32
and they'll probably want supplies to continue, but in the end a Humvee is not an APC. And a metal box without a big gun is not the same as a metal box with a big gun.
59:41
But perhaps the biggest question if you are trying to project the future of the Ukrainian military equipment park in 2024 and 2025
59:48
is what happens to the pipeline from here? A lot of countries have announced deliveries for the second half of this year and budgets for next year.
59:56
But what we don't know is how elections and politics might see those budgets evolve in allied capitals,
1:00:01
and what the plans are to convert some of that budget into equipment on the ground. The Ukrainian military has already overcome a number
1:00:08
of logistics challenges in beginning to adopt NATO equipment. There's now better training and infrastructure in place to support
1:00:14
Western systems like the Bradleys or various self-propelled guns. But the future trajectory of the force and whether it can ultimately outlast
1:00:21
Russia's own equipment reserves will probably be shaped not just by when the already pledged equipment arrives, but by what comes next.
1:00:28
And so observers will probably remain laser focused on watching the pipeline as they try and assess where and if Ukraine will get the next 500 tanks, 1,000 infantry fighting vehicles,
1:00:38
and all of the other equipment it's going to want to have as this war carries on. And so while the Ukrainian military has clearly demonstrated
1:00:45
that after almost 2.5 years of fighting they are still very much capable of fighting hard defensive actions against the Russian force,
1:00:51
it's absolutely showing the signs of the struggle as it tries to regenerate and adapt to NATO equipment in the middle of a full-scale war.
1:00:58
Going forward, only time and allied political decisions [determine] whether it's the Ukrainian or Russian equipment inventory that ultimately starts to break down first.
Channel Update
1:01:08
And OK, channel update to close out. Here my main notes are that this video essentially functions as a second part to my previous video on Russian equipment losses.
1:01:16
So if you haven't reviewed that one yet, please consider doing so. Secondly to the patrons, you should expect to see a poll going up, probably tomorrow,
1:01:23
where I'm going to be seeking votes and comments on what charitable causes the channel should aim to support in the second half of this year.
1:01:30
Finally, Perun Gaming fans I know you have been waiting a long while, but there has finally been some new content posted over there.
1:01:36
And I hope to return to Terra Invicta in the next couple of weeks as well. As always, my very best thanks to all of you
1:01:41
watching and supporting the channel, and of course to Private Internet Access for their sponsorship today. And I do hope to see you all again next week."""

In [20]:
prompt = [{"role":"system","content":"You are an AI assistant that helps extract key points out of a video transcript. \
           The user will describe what he wants you to find."},
          {"role":"user","content":f"Tell me how long Ukraine can sustain the war based on information in this video. \
           Tell me what the most important bottlenecks and shortages are that Ukraine is facing.\
           BE SURE TO ADD NUMBERS AND PERCENTAGES TO YOUR ANSWER, as many as are POSSIBLE.\
           transcript of the video: {transcript}"}]

In [17]:
completion = client.chat.completions.create(
        model='gpt-4o',
        messages=prompt,
        temperature=0.3,
        top_p=0.7,
        max_tokens=800,
        stop=None,
        stream=False
        )

print(completion.choices[0].message.content)

Based on the information provided in the video transcript, here are the key points regarding how long Russia can sustain the war, the most important bottlenecks, and shortages Russia is facing, with as many numbers and percentages as possible:

### Duration Russia Can Sustain the War
1. **Equipment Depletion**: Russia has been heavily reliant on its Soviet-era stockpiles to sustain its war effort. As of the latest data, significant portions of these stockpiles have been depleted:
   - **T-80 Tanks**: Of the roughly 1,200 T-80Bs and BVs counted in 2021, only about 200 in decent condition remain in storage.
   - **D-30 Artillery**: The number of D-30s in storage dropped from about 3,200 to 937.
   - **Overall Tank Storage**: For some tank classes, up to 50% or more of the tanks in decent condition have already been removed from storage.

2. **Production Rates**: Russia has scaled up production but still faces challenges:
   - **T-90M Tanks**: Estimates range from 231 to 267 tanks produce

In [21]:
completion = client.chat.completions.create(
        model='gpt-4o',
        messages=prompt,
        temperature=0.3,
        top_p=0.7,
        max_tokens=800,
        stop=None,
        stream=False
        )

print(completion.choices[0].message.content)

Based on the information provided in the video transcript, here are the key points regarding how long Ukraine can sustain the war, and the most important bottlenecks and shortages it is facing:

### Sustainability of the War:
1. **Equipment Losses and Resupply**:
   - Ukraine has lost nearly 1,000 infantry fighting vehicles, more than 860 tanks, around 600 artillery systems, and 500 APCs.
   - Despite these losses, Ukraine has received significant resupply from allies, including about 900 tanks, more than 1,000 infantry fighting vehicles, 3,500 armored personnel carriers, and over 5,000 infantry mobility vehicles.
   - Captured Russian equipment has also played a significant role, with Ukraine visually confirmed to have captured about 3,000 pieces of heavy equipment, including more than 500 tanks and 616 IFVs.

2. **Manpower**:
   - Ukraine has faced significant manpower shortages, but changes to mobilization rules and initiatives like the "Polish Legion" have increased induction rates

In [7]:
response_russia = """Based on the information provided in the video transcript, here are the key points regarding how long Russia can sustain the war, the most important bottlenecks, and shortages Russia is facing, with as many numbers and percentages as possible:

### Duration Russia Can Sustain the War
1. **Equipment Depletion**: Russia has been heavily reliant on its Soviet-era stockpiles to sustain its war effort. As of the latest data, significant portions of these stockpiles have been depleted:
   - **T-80 Tanks**: Of the roughly 1,200 T-80Bs and BVs counted in 2021, only about 200 in decent condition remain in storage.
   - **D-30 Artillery**: The number of D-30s in storage dropped from about 3,200 to 937.
   - **Overall Tank Storage**: For some tank classes, up to 50% or more of the tanks in decent condition have already been removed from storage.

2. **Production Rates**: Russia has scaled up production but still faces challenges:
   - **T-90M Tanks**: Estimates range from 231 to 267 tanks produced over a little more than two years, with some sources suggesting up to 200 per annum.
   - **Artillery Munitions**: In 2024, Russia is estimated to produce 1,325,000 152mm shells and 800,000 122mm shells. Rocket artillery production has also significantly increased.

3. **Foreign Aid and Imports**: Russia has been relying on foreign sources like North Korea and Iran for ammunition and possibly other military supplies.

### Bottlenecks and Shortages
1. **Quality of Equipment**: The quality of equipment being fielded has deteriorated over time:
   - **Modern vs. Old Tanks**: In early 2022, nearly 70% of visually-confirmed Russian tank losses were relatively modern designs. By mid-2024, this dropped to around 30%, with older models and wartime modifications making up a larger share.
   - **Infantry Fighting Vehicles (IFVs)**: The share of old BMP-1s in losses increased from just over 10% in early 2022 to more than 40% in mid-2023, before dropping back to around 30% in mid-2024.

2. **Storage Depletion**: Significant portions of Russia's stored equipment have been depleted:
   - **T-80 Tanks**: Only about 200 out of 1,200 T-80Bs and BVs remain in decent condition.
   - **Artillery**: More than 1,000 artillery pieces have been removed from a single storage base, reducing the number from around 1,870 to 755.

3. **Industrial Bottlenecks**: Russia faces bottlenecks in upgrading and maintaining equipment:
   - **Wartime Upgrades**: Only a small minority of tanks pulled from storage receive wartime upgrade packages. For example, only 36 out of 570 T-80BVs on the loss list had the wartime upgrade package.

4. **Diverse Equipment Quality**: The disparity in equipment quality among units is significant:
   - **T-90M vs. Older Models**: Tankers might have a 10% chance of getting a modern T-90M or end up with a tank from the 1950s or 60s.

5. **Artillery Systems**: The use of older artillery systems like the D-30 and M-46 indicates a shortage of more modern systems:
   - **D-30 Artillery**: The number of D-30s in storage has dropped significantly, and their use has increased.
   - **M-46 Artillery**: These older systems are being reactivated, likely due to shortages in other calibers and systems.

### Summary
Russia's ability to
"""

response_ukraine = """Based on the information provided in the video transcript, here are the key points regarding how long Ukraine can sustain the war, and the most important bottlenecks and shortages it is facing:

### Sustainability of the War:
1. **Equipment Losses and Resupply**:
   - Ukraine has lost nearly 1,000 infantry fighting vehicles, more than 860 tanks, around 600 artillery systems, and 500 APCs.
   - Despite these losses, Ukraine has received significant resupply from allies, including about 900 tanks, more than 1,000 infantry fighting vehicles, 3,500 armored personnel carriers, and over 5,000 infantry mobility vehicles.
   - Captured Russian equipment has also played a significant role, with Ukraine visually confirmed to have captured about 3,000 pieces of heavy equipment, including more than 500 tanks and 616 IFVs.

2. **Manpower**:
   - Ukraine has faced significant manpower shortages, but changes to mobilization rules and initiatives like the "Polish Legion" have increased induction rates.
   - The Polish Legion initiative alone saw several thousand registrations within days of its announcement.

3. **Air Force**:
   - Ukraine's Air Force has lost 97 aircraft but has received or been pledged 130 replacements.
   - The arrival of F-16s, with 95 pledged, is crucial for sustaining air capabilities, although delivery timelines extend to 2027.

### Bottlenecks and Shortages:
1. **Heavy Equipment**:
   - Despite resupply, Ukraine is roughly treading water in terms of tanks and infantry fighting vehicles. The country has mobilized new brigades that require equipment, but there isn't enough to go around.
   - Ukraine has received about 900 tanks, but only about 600 have been delivered. The visually confirmed losses are roughly 870 tanks.
   - For infantry fighting vehicles, Ukraine has lost about 830 Soviet IFVs but received 670 replacements. For NATO IFVs, losses are 137 against 729 received.

2. **Aircraft**:
   - The Ukrainian Air Force is in a vulnerable position, with significant losses in Sukhoi 24 and 25 aircraft. The F-16s are expected to replace many of these losses, but delivery timelines are long.
   - The MiG-29 fleet is treading water with 30 losses and 27 replacements, but the number of tasks assigned to this fleet has increased.

3. **Light Vehicles**:
   - Ukraine has received a significant number of light vehicles like MRAPs and Humvees, but these are not ideal substitutes for heavier equipment in offensive operations.
   - The visually confirmed loss of fewer than 300 NATO APCs has been balanced by inflows of more than 3,300, resulting in a net positive of 3,060 vehicles.

4. **Domestic Production and Reactivation**:
   - There is limited public data on how much equipment Ukraine is producing or reactivating domestically. However, it is happening, and budget is being allocated to it.

5. **Captured Equipment**:
   - While captured Russian equipment has been significant, the rate of capturing new equipment has decreased, making Ukraine more dependent on domestic production and allied deliveries.

### Numbers and Percentages:
- **Tanks**: 870 losses, 900 pledged, 600 delivered.
- **IFVs**: 830 Soviet losses, 670 replacements; 137 NATO losses, 729 replacements.
- **Aircraft**: 97 losses, 130 replacements pledged.
- **Captured Equipment**: 3,000 pieces, including 500 tanks and 616 IFVs.
- **Light Vehicles**: 300 NATO APC losses, 3,300 replacements, net positive 3,060.

### Conclusion:
Ukraine can sustain the war as long as the resupply from allies continues and domestic production can be ramped up. However, the most critical bottlenecks are
"""

In [8]:
prompt = [{"role":"system","content":"You are an AI assistant that helps the user."},
          {"role":"user","content":f"Compare the points in the text about Ukrainian and Russian attrition losses in the war so far. Tell me which side is better off, and maybe how long until they ecounter significant problems.\
           . Russia: {response_russia}; Ukraine: {response_ukraine}\
            Do not just repeat the information, but analyze and compare the two sides."}]

In [29]:
completion = client.chat.completions.create(
        model='gpt-4o',
        messages=prompt,
        temperature=0.3,
        top_p=0.7,
        max_tokens=4096,
        stop=None,
        stream=False
        )

print(completion.choices[0].message.content)

### Comparative Analysis of Ukrainian and Russian Attrition Losses

#### Equipment Depletion and Resupply

**Russia:**
- **Tanks and Artillery:** Russia has significantly depleted its Soviet-era stockpiles. For example, only about 200 out of 1,200 T-80Bs and BVs remain in decent condition. Artillery stockpiles have also seen a drastic reduction, with D-30s in storage dropping from about 3,200 to 937.
- **Production Rates:** Russia has increased production rates but still faces challenges. For instance, T-90M tank production estimates range from 231 to 267 tanks over two years, with some sources suggesting up to 200 per annum. Artillery munitions production has also increased.
- **Foreign Aid:** Russia is relying on foreign sources like North Korea and Iran for ammunition and possibly other military supplies.

**Ukraine:**
- **Tanks and IFVs:** Ukraine has lost nearly 1,000 infantry fighting vehicles and more than 860 tanks. However, it has received significant resupply from allies, inc

In [37]:
import os
from genai_sdk import PwCAzureChatOpenAI
from dotenv import load_dotenv

# Construct the full path to the env file
directory = os.getcwd()
file_name = 'conf.env'  # Replace with the name of file
file_path = os.path.join(directory, file_name)

# Load environment variables from .env file (for a service account --> set override = True)
load_dotenv(file_path)

# Get the API_KEY
api_key = os.getenv('API_KEY')

llm = PwCAzureChatOpenAI(
    apikey=api_key,
    api_version="2023-03-15-preview",
    model='gpt-4o',
    max_tokens=4096,
    temperature=0.7
)

print(llm.invoke('Tell me a joke about synchronous function!').content)

Why did the synchronous function go to therapy?

Because it couldn't handle the wait!


In [11]:
from openai import AzureOpenAI

KEY = "07d376b4bbce4baf8a746f8be5437d0d"
ENDPOINT = "https://lucasaieastus2servicesopen.cognitiveservices.azure.com/"

client = AzureOpenAI(
        api_key=KEY,
        api_version="2023-12-01-preview",
        azure_endpoint=ENDPOINT,
    )

In [70]:
completion = client.chat.completions.create(
        model='gpt-4o',
        messages=[
            {
                "role": "system",
                "content": "You are an AI assistant that helps the user."
            },
            {
                "role": "user",
                "content": "Tell me a joke about synchronous function!"
            }
        ],
        temperature=0.3,
        top_p=0.7,
        max_tokens=4096,
        stop=None,
        stream=False
        )

print(completion.choices[0].message.content)

Why did the synchronous function go to therapy?

Because it couldn't handle the wait!
